In [1]:
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/Bertopic/shared_work/'
dataset_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/genius_and_wasabi/'

# # This will mount your google drive under 'MyDrive'
# drive.mount('/content/gdrive', force_remount=True)
# # In order to access the files in this notebook we have to navigate to the correct folder
# os.chdir(gdrive_path)
# # Check manually if all files are present
# print(sorted(os.listdir()))


# To run from the common drive:
dataset_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/genius_and_wasabi/'
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Common Drive instructions:

!pip install pandas numpy bertopic
!pip install gensim nltk matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.5 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039159 sha256=2fd4486d7141cb4b251d2d4e1fc9bae292ccd75b8c7bb8b1ed1f1ec8b25df076
  Stored in di

In [3]:
# !pip install pandas bertopic

In [4]:
# pip install pandas nltk

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from umap import UMAP

# set path of the file
# csv_file_path = 'chunk_5.csv'

# Common drive dataset path - change chunk/sample accordingly
csv_file_path = dataset_path + 'chunk_5.csv'

df = pd.read_csv(csv_file_path)

In [8]:
# Model path
# model_path = '/content/gdrive/MyDrive/Bertopic/shared_work/mymodel'

# Model path for common drive
model_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_' + csv_file_path.split('/')[-1].split('.')[0]

print(model_path)

/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_chunk_5


 If you are loading a pre-trained model, you should use transform() instead of fit_transform() to get the topics for new documents or fit_transform() if you are updating the model with new documents.

In [9]:
# Removing explicit song structure as it is not important information
import re

def remove_explicit_song_structure(lyrics):
  pattern = r'\[.+?\]'
  cleaned_lyrics = re.sub(pattern, '', lyrics)
  return cleaned_lyrics

df['cleaned_lyrics'] = df['lyrics'].astype(str).apply(remove_explicit_song_structure)
df.head()

,Unnamed: 0,title,genre,artist,year,views,features,lyrics,language,gender,cleaned_lyrics
0,10,Dont Look Over Your Shoulder,rb,Nancy Wilson,1967,15,{},Don't look\nDon't look\nDon't look\nDon't look...,en,Female,Don't look\nDon't look\nDon't look\nDon't look...
1,11,This Mothers Daughter,rb,Nancy Wilson,1976,174,{},[Verse 1]\nLies beyond her years\nThis mother'...,en,Female,\nLies beyond her years\nThis mother's daughte...
2,12,Changes,rb,Nancy Wilson,1977,15,{},[Verse 1]\nIf you love me the way you say\nThi...,en,Female,\nIf you love me the way you say\nThink about ...
3,13,Patience My Child,rb,Nancy Wilson,1977,19,{},"[Verse 1]\nWhen I was a youngster, I can recal...",en,Female,"\nWhen I was a youngster, I can recall\nSeekin..."
4,14,Nobody,rb,Nancy Wilson,1977,13,{},[Verse 1]\nNobody knows me like you do\nNobody...,en,Female,\nNobody knows me like you do\nNobody feels my...


In [10]:
df['has_lyrics'] = ~df['lyrics'].isna() # flag the rows that have lyrics
documents = df[df['has_lyrics']]['cleaned_lyrics'].astype(str).tolist()

In [11]:
# loaded_model = BERTopic.load("/content/gdrive/MyDrive/Bertopic/shared_work/mymodel")

loaded_model = BERTopic.load(model_path)
topics, prob = loaded_model.transform(documents)
# df.loc[~df['has_lyrics'], 'topic'] = "No Lyrics"
# df.loc[df['has_lyrics'], 'topic'] = topics
print(topics)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

[168  10   2 ...  12   0 160]


In [13]:
generated_topics = loaded_model.topics_
# print(len(topics))
topic_labels = loaded_model.generate_topic_labels()
topic_labels

['-1_love_dont_know',
 '0_nigga_niggas_bitch',
 '1_jesus_lord_praise',
 '2_feel_im_life',
 '3_christmas_santa_merry',
 '4_said_john_went',
 '5_dance_party_shake',
 '6_mi_yuh_di',
 '7_baby_love_baby baby',
 '8_shes_shell_shes hair',
 '9_america_war_people',
 '10_shes_shell_shes got',
 '11_sea_ship_sail',
 '12_na_susan aint_aint susan',
 '13_blues_blue_blu',
 '14_love_unpredictable_love unpredictable',
 '15_dream_world_light',
 '16_fool_slipped_slipped tripped',
 '17_bye bye_bye_mississippi',
 '18_falls_ahahah_falls falls',
 '19_unbreak_unbreak heart_cried',
 '20_christ_let adore_bethlehem',
 '21_rain_raining_rainy',
 '22_la_la la_lalalalalala',
 '23_man_woman_need man',
 '24_moon_moonlight_blue moon',
 '25_away far_far away_far',
 '26_angel_youre angel_angels',
 '27_ruins_white flag_hanging line',
 '28_remember_tat_tat tat',
 '29_hey_hey hey_way way',
 '30_autumn_summer_love paris',
 '31_lyrics snippet_snippet_lyrics',
 '32_goodbye_sweet day_day sweet',
 '33_limb_limb limb_dont dont',
 

In [14]:
df.loc[~df['has_lyrics'], 'topic'] = "No Lyrics"
df.loc[df['has_lyrics'], 'topic'] = generated_topics

# Group by topics and gender, and count occurrences, aggregate lyrices by topic and gender
topic_gender_distribution = df.groupby(['topic', 'gender']).size().unstack(fill_value=0)

#  normalize the counts to compare proportions rather than raw counts
topic_gender_distribution_normalized = topic_gender_distribution.div(topic_gender_distribution.sum(axis=1), axis=0)

In [15]:
df.head()

,Unnamed: 0,title,genre,artist,year,views,features,lyrics,language,gender,cleaned_lyrics,has_lyrics,topic
0,10,Dont Look Over Your Shoulder,rb,Nancy Wilson,1967,15,{},Don't look\nDon't look\nDon't look\nDon't look...,en,Female,Don't look\nDon't look\nDon't look\nDon't look...,True,14
1,11,This Mothers Daughter,rb,Nancy Wilson,1976,174,{},[Verse 1]\nLies beyond her years\nThis mother'...,en,Female,\nLies beyond her years\nThis mother's daughte...,True,-1
2,12,Changes,rb,Nancy Wilson,1977,15,{},[Verse 1]\nIf you love me the way you say\nThi...,en,Female,\nIf you love me the way you say\nThink about ...,True,-1
3,13,Patience My Child,rb,Nancy Wilson,1977,19,{},"[Verse 1]\nWhen I was a youngster, I can recal...",en,Female,"\nWhen I was a youngster, I can recall\nSeekin...",True,-1
4,14,Nobody,rb,Nancy Wilson,1977,13,{},[Verse 1]\nNobody knows me like you do\nNobody...,en,Female,\nNobody knows me like you do\nNobody feels my...,True,-1


In [17]:
outlier_docs = df[df['topic'] == -1]
len(outlier_docs)

27138

In [19]:
new_topics = loaded_model.reduce_outliers(documents, topics)

In [21]:
type(new_topics)

list

In [22]:
df.loc[~df['has_lyrics'], 'topic'] = "No Lyrics"
df.loc[df['has_lyrics'], 'topic_without_outliers'] = new_topics

df.head()

,Unnamed: 0,title,genre,artist,year,views,features,lyrics,language,gender,cleaned_lyrics,has_lyrics,topic,topic_without_outliers
0,10,Dont Look Over Your Shoulder,rb,Nancy Wilson,1967,15,{},Don't look\nDon't look\nDon't look\nDon't look...,en,Female,Don't look\nDon't look\nDon't look\nDon't look...,True,14,14.0
1,11,This Mothers Daughter,rb,Nancy Wilson,1976,174,{},[Verse 1]\nLies beyond her years\nThis mother'...,en,Female,\nLies beyond her years\nThis mother's daughte...,True,-1,10.0
2,12,Changes,rb,Nancy Wilson,1977,15,{},[Verse 1]\nIf you love me the way you say\nThi...,en,Female,\nIf you love me the way you say\nThink about ...,True,-1,109.0
3,13,Patience My Child,rb,Nancy Wilson,1977,19,{},"[Verse 1]\nWhen I was a youngster, I can recal...",en,Female,"\nWhen I was a youngster, I can recall\nSeekin...",True,-1,206.0
4,14,Nobody,rb,Nancy Wilson,1977,13,{},[Verse 1]\nNobody knows me like you do\nNobody...,en,Female,\nNobody knows me like you do\nNobody feels my...,True,-1,243.0


In [25]:
df['topic'].value_counts()

-1      27138
 0       3427
 1        602
 2        579
 3        519
        ...  
 256       10
 257       10
 258       10
 259       10
 260       10
Name: topic, Length: 262, dtype: int64

In [23]:
df['topic_without_outliers'].value_counts()

 2.0      6396
 0.0      5528
 7.0      4126
 14.0     1706
 1.0      1213
          ... 
 212.0      14
 216.0      14
 237.0      14
 238.0      13
-1.0         1
Name: topic_without_outliers, Length: 262, dtype: int64

In [28]:
loaded_model.update_topics(docs=documents, topics=new_topics)

2023-12-17 21:55:58,713 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


Visualizing topic distance after reducing outliers

In [29]:
loaded_model.visualize_topics()

In [31]:
loaded_model.visualize_documents(documents)

In [30]:
labels = loaded_model.generate_topic_labels()
labels

['-1_mine_are_one',
 '0_nigga_shit_bitch',
 '1_lord_jesus_god',
 '2_to_feel_of',
 '3_christmas_santa_year',
 '4_he_his_said',
 '5_dance_party_shake',
 '6_mi_di_yuh',
 '7_baby_love_oh',
 '8_she_her_loved',
 '9_america_people_war',
 '10_she_her_he',
 '11_ship_sea_captain',
 '12_ride_gonna_na',
 '13_blues_blue_mr',
 '14_love_heart_you',
 '15_dream_light_world',
 '16_fool_he_slipped',
 '17_bye_country_mississippi',
 '18_coming_falls_ghost',
 '19_tears_cried_un',
 '20_sing_born_king',
 '21_rain_sun_raining',
 '22_la_sha_da',
 '23_man_woman_he',
 '24_moon_moonlight_stars',
 '25_far_away_miles',
 '26_angel_an_angels',
 '27_flag_our_beware',
 '28_remember_forget_tat',
 '29_hey_eh_way',
 '30_spring_summer_autumn',
 '31_snippet_lyrics_from',
 '32_goodbye_say_together',
 '33_limb_gave_giving',
 '34_tomorrow_meet_strangers',
 '35_pass_sun_color',
 '36_pretty_bigger_sweeter',
 '37_road_long_ease',
 '38_work_money_mojo',
 '39_kiss_lips_soul',
 '40_secret_school_special',
 '41_sunshine_sunny_happy',
